In [5]:
from data.dataset import CheXpertDataset
dataset = CheXpertDataset(
    csv_file="./data/valid.csv",
    root_dir="../image_data/",
)

In [6]:
test = dataset.__getitem__(0)
print(test)

{'image': '../image_data/CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg', 'sex': 'Male', 'age': np.int64(73), 'frontal/lateral': 'Frontal', 'ap/pa': 'AP', 'no_finding': np.float64(0.0), 'enlarged_cardiomediastinum': np.float64(1.0), 'cardiomegaly': np.float64(1.0), 'lung_opacity': np.float64(1.0), 'lung_lesion': np.float64(0.0), 'edema': np.float64(0.0), 'consolidation': np.float64(0.0), 'pneumonia': np.float64(0.0), 'atelectasis': np.float64(0.0), 'pneumothorax': np.float64(0.0), 'pleural_effusion': np.float64(0.0), 'pleural_other': np.float64(0.0), 'fracture': np.float64(0.0), 'support_devices': np.float64(0.0)}
